#get data 

In [358]:
import numpy as np
import pandas as pd

train_data_pd = pd.read_csv("winequality-white.csv", delimiter=";")
train_data = train_data_pd.values

test_data_pd = pd.read_csv("winequality-red.csv", delimiter=";")
test_data = test_data_pd.values
test_labels = test_data_pd.columns
print(test_labels)

test_data_pd2 = pd.read_csv("iris.csv", skip_blank_lines=True)
test_data2 = test_data_pd2.values
test_data2 = test_data2[:-2]
labels_iris = ['sepal length' , 'sepal width' , 'petal length', 'petal width' ,'class']
print(test_data2.shape)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')
(149, 5)


In [282]:
def check_purity(data):
    #check if all quality values are the same
    quality_column = data[:, -1]
#     print(quality_column)
#     print(quality_column)
#     unique_classes = set(quality_column)
#     print(unique_classes)
#     print(len(unique_classes))
    unique_classes, class_counts = np.unique(quality_column, return_counts=True)
#     print(class_counts)
    class_probabilities = class_counts / len(data)
    
    return max(class_probabilities)
    
#     if max(class_probabilities) >= min_purity:
#         return True
#     else:
#         return False

In [359]:
# #test check purity
# print(train_data)
# print(check_purity(train_data))

In [227]:
def classify_data(data):
    #find the quality that occurs the most
    quality_column = data[:, -1]
    
    unique_classes, counts = np.unique(quality_column, return_counts=True)
#     print(unique_classes)
#     print(counts_unique_classes)
    i = counts.argmax()
    qual = unique_classes[i]
#     print(index)
#     print(qual, counts[i])
    
    return qual, counts[i]

In [228]:
# test classify data
print(classify_data(test_data))

(5.0, 681)


In [191]:
def get_potential_splits(data):
    
    potential_splits = {}
    n_columns = data.shape
    n_columns = n_columns[1]
#     print(data.shape, "shape")
    for i in range(n_columns - 1):       
        values = data[:, i]
        unique_values = np.unique(values)
        #print(unique_values)
        
    
        if len(unique_values) > 1:
            potential_splits[i] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
#                     previous_value = unique_values[index ]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[i].append(potential_split)
    
    return potential_splits

In [49]:
# test potential splits
#print(get_potential_splits(test_data))

In [56]:
def split_data(data, col, val):
    
    split = data[:, col]
    #print(split_column_values)

    below = data[split <= val]
    above = data[split >  val]
    
    return below, above


In [58]:
# test split_data
# print(split_data(test_data, 0, 7.7))

In [59]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [60]:
print(calculate_entropy(test_data))

1.7090616375059242


In [61]:
def calculate_overall_entropy(below, above):
    
    n = len(below) + len(above)
    p_data_below = len(below) / n
    p_data_above = len(above) / n

    overall_entropy =  (p_data_below * calculate_entropy(below) 
                      + p_data_above * calculate_entropy(above))
    
    return overall_entropy

In [193]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, col=column_index, val=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [496]:
def decision_tree_algorithm(data,pd, min_samples=2, min_purity = 1, indent = ""):
    p = check_purity(data)
    if (check_purity(data) >= min_purity) or (len(data) < min_samples):
        classification, occur = classify_data(data)
#         print("p:", p)
#         print("class:", classification)
#         print("occur:" , occur)
        return classification, occur, p
    else:

        # helper functions 
#         print(data.shape, "shape")
        potential_splits = get_potential_splits(data)
        col, val = determine_best_split(data, potential_splits)
#         print(col, val)
        data_below, data_above = split_data(data, col, val)
        
        # determine question
        feature_name = pd[col]
#         type_of_feature = FEATURE_TYPES[split_column]
    
        _, occur = classify_data(data)
        q1 = "{} <= {}".format(feature_name, val)
        question = (q1, occur)
#         print(indent+'|'+ question)

#         question = "{} = {}".format(feature_name, split_value)
        
        
        sub_tree = {question: []}
        
        y = decision_tree_algorithm(data_below,pd,  min_samples, indent = indent + "   ")
        n = decision_tree_algorithm(data_above,pd, min_samples, indent = indent + "   ")
        sub_tree[question].append(y)
        sub_tree[question].append(n)
#         if yes_answer == no_answer:
# #             sub_tree[question].append(yes_answer)
#             if (yo != -1):
#                 print(indent +  "   "+ "leaf: class="+str(yes_answer)+", p="+str(yp)+", size="+ str(yo))
# #                 print(indent, yes_answer, yo, yp)
            
# #             sub_tree = yes_answer
#         else:
# #             sub_tree[question].append(yes_answer)
# #             sub_tree[question].append(no_answer)
#             if (yo != -1):
#                 print(indent+  "   " + "leaf: class="+str(yes_answer)+", p="+str(yp)+", size="+ str(yo))
# #                 print(indent, yes_answer, yo, yp)
#             if(no != -1):
#                 print(indent+  "   " + "leaf: class="+str(no_answer)+", p="+str(np)+", size="+ str(no))
            

        
        return sub_tree, -1, -1

In [532]:
def test(example, tree, labels):
    key = list(tree.keys())[0]
    question = list(tree.keys())[0][0]
#     print(key)
    name,  value = question.split(" <= ")
#     print(name)
#     print('\n', tree)
    if example[labels[name]] <= float(value):
        answer = tree[key][0][0]
    else:
        answer = tree[key][1][0]

    if not isinstance(answer, dict):
#         print(answer)
        return answer
    
    else:
        residual_tree = answer
        return test(example, residual_tree, labels)

In [529]:
def accuracy(test_data, tree, labels):
    t = 0
    c = 0
    for i in test_data:
        out = test(i, tree, labels)
        if(out == i[-1]):
            c+=1
        t+=1
    return c/t
            

In [539]:
def print_tree(tree, indent = ""):
#     print(tree)
#     print(type(tree))
    if(isinstance(tree, dict)):
        key = next(iter(tree.keys()))
        
        print(indent+key[0]+", size="+str(key[1]))
        val = next(iter(tree.values()))
#         print('val',val)
        print_tree(val, indent = indent + "    ")
    if(isinstance(tree, list)):
        for i in tree:
            if(isinstance(i[0], dict)):
                print_tree(i[0], indent = indent)
            else:
#                 print(i)
                print(indent+"leaf: class="+str(i[0])+", p="+str(i[1])+", size="+str(i[2]))   

In [533]:
#test with iris
tree = decision_tree_algorithm(test_data2, labels_iris, min_samples = 5, min_purity=0.98)
# print(tree)
print_tree(tree[0])
labels = {}
c = 0
for i in labels_iris:
    labels[i] = c
    c += 1
accuracy(test_data2, tree[0], labels)

petal width <= 0.8, size=50
    leaf: class=Iris-setosa, p=49, size=1.0
    petal width <= 1.75, size=50
        petal length <= 4.95, size=49
            petal width <= 1.65, size=47
                leaf: class=Iris-versicolor, p=47, size=1.0
                leaf: class=Iris-virginica, p=1, size=1.0
            petal width <= 1.55, size=4
                leaf: class=Iris-virginica, p=3, size=1.0
                leaf: class=Iris-versicolor, p=2, size=0.6666666666666666
        petal length <= 4.85, size=45
            leaf: class=Iris-virginica, p=2, size=0.6666666666666666
            leaf: class=Iris-virginica, p=43, size=1.0


0.9865771812080537

# WINE DATA TREE

In [540]:

tree = decision_tree_algorithm(train_data, test_labels)
labels = {}
print_tree(tree[0])
c = 0
for i in test_labels:
    labels[i] = c
    c += 1
print(accuracy(test_data, tree[0], labels))



alcohol <= 10.850000000000001, size=2198
    volatile acidity <= 0.2525, size=1353
        volatile acidity <= 0.2075, size=788
            density <= 0.99788, size=400
                sulphates <= 0.535, size=352
                    residual sugar <= 2.1500000000000004, size=235
                        free sulfur dioxide <= 13.5, size=78
                            chlorides <= 0.0335, size=12
                                leaf: class=5.0, p=9, size=1.0
                                sulphates <= 0.46499999999999997, size=3
                                    alcohol <= 9.7, size=3
                                        leaf: class=3.0, p=1, size=1.0
                                        leaf: class=6.0, p=3, size=1.0
                                    leaf: class=5.0, p=3, size=1.0
                            fixed acidity <= 7.95, size=75
                                density <= 0.99162, size=58
                                    alcohol <= 10.75, size=8
                 